<a href="https://colab.research.google.com/github/ThanhHung2112/LMS/blob/main/t5_grammar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Install libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q datasets tqdm pandas
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... do

In [ ]:
# !pip install dask

In [ ]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
# Check we have a GPU and check the memory size of the GPU
!nvidia-smi

Mon Oct  9 16:02:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Import packages**

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import random
import numpy as np
import torch
import datasets

In [ ]:
import tensorflow_datasets

In [ ]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

# ***C4-200M dataset***

In [ ]:
# remove temp
# rm -r /tmp/

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/c4_200m/c4_200m_550k.csv')
# df = pd.read_csv('/content/c4data.csv')
len_df = 18386520  # C4_200M.tsv-00000-of-00010
start =  1838651 # 919325 +
end = int(18386520/20)

In [ ]:
start = 4000000
end = 4500000

In [ ]:
print(start, end, end - start)

4000000 4500000 500000


In [ ]:
# Unzip the data file
def unzip(zip_file=None):
    try:
        with zipfile.ZipFile(zip_file) as z:
            z.extractall("./")
            print("Extracted all")
    except:
        print("Invalid file")

# unzip('/content/drive/MyDrive/LMS_Document/Code/t5_gec_model_03.zip')

In [ ]:
# , skiprows=1000000, nrows=999999)
# df_main = pd.read_csv('/kaggle/input/c4200m/C4_200M.tsv-00000-of-00010', delimiter='\t', skiprows=start, nrows=end) # on_bad_lines='skip
df_main = pd.read_csv('/content/drive/MyDrive/LMS_Document/Code/C4_200M.tsv-00000-of-00010',delimiter='\t',skiprows=start, nrows=end)

In [ ]:
df_main.columns = ["input", "target"]

In [ ]:
df = df_main

In [ ]:
df = df_main.iloc[4000000:4300000]
df # 919325 + 1838651 -- 100000 -- 3419332 4419332 1000000 --

,input,target
4000000,"SANDYS, Samuel (1560-1623), this of Ombersley, Worcs.","SANDYS, Samuel (1560-1623), of Ombersley, Worcs."
4000001,"Walnut Hair Color 187799 32 Likes 7 Ments San Diego Hairstylist Kristinareneehair On Uploaded by Caress Robert on Friday November 16th, 2018 in category Hair Color.","Walnut Hair Color 187799 32 Likes 7 Ments San Diego Hairstylist Kristinareneehair On Uploaded by Caress Robert on Friday, November 16th, 2018 in category Hair Color."
4000002,"Tuners These are also a small thing, but rarely they are collected and needed for steel strings.","Tuners These are also a small thing, but rarely supplied and are necessary for steel strings."
4000003,"haemoglobinuria (Brietischer Journal: hematology?, 89, 79-82.","haemoglobinuria. British Journal of Haematology, 89, 79–82."
4000004,"BTITF has the right of being in the Eurojumelages board, and this year I was elected as Deputy Treasurer.","BTITF has the right of being in the Eurojumelages board, and this year I was elected as Deputy Treasurer."
...,...,...
4299995,"Thanks, Jen there's so much for process that I still don't see an end in sight :) Let me know if you need more information about ProGen I really learned a lot from going through the program.","Thanks, Jen. There's so much to process that I still don't see an end in sight :) Let me know if you need more information about ProGen. I really learned a lot from going through the program."
4299996,The cost per ticket range from 500Euro forS for a full access to conference-only tickets for 250 Euro you also can get tutorial-only pass if you only like to get-in for those sessions (for 295 Euro) conference is happens in 11th June through 13th so be sure to receive your ticket quickly and reservation your spot now!,The cost per ticket ranges from 500 Euro for a full access pass to conference-only tickets for 250 Euro. You can also get a tutorial-only pass if you'd only like to get in on those sessions (for 295 Euro). The conference happens June 11th through 13th so be sure to get your tickets quickly and reserve your spot now!
4299997,"Describing about Deciduous: Berberis thunbergii Rosy Rocket PP18,413""","description for ""Deciduous: Berberis thunbergii 'Rosy Rocket' PP18,413"""
4299998,Where The tournament were place 13:00 25th of Feb in house 7.,Where? The tournament will take place 13:00 25th of Feb in house 7.


In [ ]:
# Hiển thị thông tin cơ bản về dataframe
print("Thông tin cơ bản về dataframe:")
print(df.info())

# Hiển thị một số hàng đầu tiên của dataframe
print("\nMột số hàng đầu tiên của dataframe:")
print(df.head())

# Kiểm tra xem có giá trị bị thiếu (NaN) trong dataframe không
print("\nKiểm tra giá trị bị thiếu trong dataframe:")
print(df.isnull().sum())


Thông tin cơ bản về dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 4000000 to 4299999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   input   300000 non-null  object
 1   target  299999 non-null  object
dtypes: object(2)
memory usage: 4.6+ MB
None

Một số hàng đầu tiên của dataframe:
                                                                                                                                                                        input  \
4000000                                                                                                                 SANDYS, Samuel (1560-1623), this of Ombersley, Worcs.   
4000001  Walnut Hair Color 187799 32 Likes 7 Ments San Diego Hairstylist Kristinareneehair On Uploaded by Caress Robert on Friday November 16th, 2018 in category Hair Color.   
4000002                                                                      Tuners These a

In [ ]:
# Loại bỏ các hàng chứa giá trị bị thiếu (NaN) trong dataframe
df.dropna(inplace=True)

print("Thông tin cơ bản sau khi loại bỏ giá trị bị thiếu:")
print(df.info())
print("\nKiểm tra giá trị bị thiếu trong dataframe sau khi loại bỏ:")
print(df.isnull().sum())

Thông tin cơ bản sau khi loại bỏ giá trị bị thiếu:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 299999 entries, 4000000 to 4299999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   input   299999 non-null  object
 1   target  299999 non-null  object
dtypes: object(2)
memory usage: 6.9+ MB
None

Kiểm tra giá trị bị thiếu trong dataframe sau khi loại bỏ:
input     0
target    0
dtype: int64


<ipython-input-17-47ff485d68f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [ ]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [ ]:
# model_name = 't5-base'
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
!unrar x /content/drive/MyDrive/LMS_Document/Code/t5_gec_model_03.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/LMS_Document/Code/t5_gec_model_03.rar

Creating    t5_gec_model_03                                           OK
Extracting  t5_gec_model_03/config.json                                    0%  OK 
Extracting  t5_gec_model_03/generation_config.json                         0%  OK 
Extracting  t5_gec_model_03/pytorch_model.bin                              0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63%

In [ ]:
# model_name = '/content/t5_gec_model_03'
model_name = '/content/drive/MyDrive/c4_200m/weights/checkpoint-1000'

# model_name = '/kaggle/input/t5-gec/kaggle/working/t5_gec_model_02'

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((269999, 2), (30000, 2))

In [ ]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (1696 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
test_df.head()

,input,target,input_token_len
4004941,"Alfred’s daughter, Susana, shot a video as her dad ran in to pull the woman from the car.","Alfred’s daughter, Susana, shot video as her dad ran in to pull the woman from the car.",26
4051775,Package - Water - gas - electricity - swimming pool and final umbrellas: - 90 Euro.,"Package - Water, gas, electricity, swimming pool and final cleaning: 90 euros.",25
4197579,On the Isle of Man itself of course!,On the Isle of Man of course!,10
4299321,I understand the cost is a one date charge of $ 977 Which is a little cheaper than that of the industry average ($ 1500 to $3000) However you must at the time of writing by a special discount and you only need pay $599) I consider this a decent price without being tried of the program first hand.,"I understand the cost is a one time charge of $ 977 which is a little lower than the industry average ($ 1500 to $3000). However, at the time of writing there is a special discount and you only need pay $599. I consider this a decent price without having tried the program first hand.",70
4083925,"She has traveled over 30 countries, know five languages and in her free time, create acrylic and watercolor paints.","She has traveled in over 30 countries, knows five languages and in her free time, creates acrylic and watercolor paintings.",24


In [ ]:
test_df['input_token_len'].describe()

count    30000.000000
mean        33.510033
std         27.235026
min          3.000000
25%         17.000000
50%         27.000000
75%         42.000000
max       1696.000000
Name: input_token_len, dtype: float64

### We will use a token length of 64 since it will cover the vast majority of examples

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
test_dataset

Dataset({
    features: ['input', 'target', '__index_level_0__'],
    num_rows: 30000
})

### Load the Dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader

class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['target'] # output

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [ ]:
dataset = GrammarDataset(train_dataset, tokenizer, True)
# dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 63
attention_mask 63
labels 61
{'input_ids': [1853, 350, 4386, 476, 3, 13011, 26479, 7454, 5, 10, 6393, 5627, 1620, 115, 6, 3, 15, 26, 5, 10, 2963, 9, 7459, 739, 5, 41, 31971, 29004, 61, 108, 6, 3, 115, 184, 210, 6, 898, 3, 635, 14156, 4271, 3, 89, 17, 5, 201, 3097, 635, 11372, 6, 2560, 3, 89, 17, 5, 61, 17189, 15, 10, 71, 6463, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [37, 12630, 9840, 162, 5, 7454, 5, 10, 6393, 5627, 1620, 115, 6, 3, 15, 26, 5, 10, 2963, 9, 7459, 739, 5, 41, 31971, 29004, 61, 108, 6, 3, 115, 184, 210, 6, 898, 3, 635, 14156, 4271, 3, 89, 17, 5, 201, 3097, 635, 11372, 6, 2560, 3, 89, 17, 5, 137, 17189, 15, 10, 71, 6463, 5, 1]}


### Define Evaluator

In [ ]:
!pip install --q rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

<ipython-input-26-76fa4d2ef6b0>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


### Train Model

In [ ]:
!pip install -q accelerate -U
!pip install -q transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
!pip install -q datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.5 MB/s eta 0:00:00


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [ ]:
# defining training related arguments
batch_size = 16
args = Seq2SeqTrainingArguments(
                        output_dir="/content/drive/MyDrive/c4_200m/weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=1,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True, # only while using CUDA
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb", # report  to wandb
    )

In [ ]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(
                model=model,
                args=args,
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
model_name = '/content/drive/MyDrive/LMS_Document/Code/t5_gec_model_04'
# model_name = '/content/drive/MyDrive/LMS_Document/Code/textmodel_ok'

trainer.save_model(model_name)

In [ ]:
# ! zip -r /content/t5_gec_model_04 . -i t5_gec_model_04/
# !zip '/content/t5_gec_model_04'


  <a href="/content/t5_gec_model_03"> Download File </a>

In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 't5_gec_model_02.zip'):
    """
    zip all the files in a directory

    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory

    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'

    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [ ]:
zip_dir()

/kaggle/working/t5_gec_model_02.zip

## Testing

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
# model_name = 'deep-learning-analytics/GrammarCorrector'
model_name = '/kaggle/working/t5_gec_model_02'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
input_text = "I am enjoys, writtings Articles ons AI and I also enjoyed write articling on AI."
num_return_sequences = 1
corrected_text = correct_grammar(input_text, num_return_sequences)
print(corrected_text)


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  logger.info(f"Configuration saved in {output_config_file}")


['I enjoy writing articles on AI and I also enjoy writing articles on AI.']


In [ ]:
text = 'They could culture more land and grows food a lot more.'
print(correct_grammar(text, num_return_sequences=3))

['They could cultivate more land and grow food a lot more.', 'They could culture more land and grow food a lot more.', 'They could grow more land and grow food a lot more.']


In [ ]:
text = 'he are an teachers'
print(correct_grammar(text, num_return_sequences=2))

['he is a teacher.', 'he is an educator.']


In [ ]:
text =  """The pie charts illustrate the number of houses owned and rented in the UK in 1991 and 2007.
Overall, homeowners were the most popular typing housing, with more than half the total over the period shown. In addition, the number of people with private renting was stable, whereas the figures for social renting and social housing experienced a decrease.
"""
print(correct_grammar(text, num_return_sequences))

['The pie charts illustrate the number of houses owned and rented in the UK in 1991 and 2007. Overall, homeowners were the most popular type of housing, with more than half the total over the period shown. In addition, the number of people with private renting was stable, whereas the figures for social renting and social']
